In [71]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import string
import nltk
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score


from subprocess import check_output

In [58]:
# Import the email modules we'll need
import glob
import email
import mailparser
from email import policy
from email.parser import BytesParser

path = '../datawe/raw/Email_Classification/*'
email_types = glob.glob(path)
appendFilesData = []
file_raw_data = [] 
for folder in email_types:
    files = glob.glob(folder+"/*.txt")
    email_type = folder.split('\\')[1]
    for name in files:
        try:
            with open(name) as fp:
                raw_data = fp.read()
                file_raw_data.append(raw_data)
                msg = mailparser.parse_from_string(raw_data)
                appendFilesData.append({
                    "to":msg.to,
                    "from":msg.from_,
                    "subject":msg.subject,
                    "date":msg.date,
#                     "sent":msg["Sent"],
#                     "importance":msg["Importance"],
                    "content":  msg.body,
                    "class_to_exec":email_type,
                })
         
        except IOError as exc:
            print('Exception')

            
#creating pandas dataframe
data = pd.DataFrame(appendFilesData)

In [173]:
##text cleaning
def clean(doc):
    words_to_exclude = set(stopwords.words('english'))
    exclude = set(string.punctuation)
    
    
    tagged_sentence = nltk.tag.pos_tag(doc.split())
    edited_sentence = ' '.join([word for word,tag in tagged_sentence if tag != 'NNP' and tag != 'NNPS'])

    word_free = " ".join([i for i in edited_sentence.lower().split() if i not in words_to_exclude])
    punc_free = ''.join(ch for ch in word_free if ch not in exclude)
    alpha_free = " ".join(word for word in punc_free.split() if word.isalpha())
    #     stemmer = SnowballStemmer("english").stem
    #     stem_free = " ".join(stemmer(stem) for stem in alpha_free.split())
    lemma = WordNetLemmatizer()
    normalized = " ".join(lemma.lemmatize(word) for word in alpha_free.split())
    
    print(edited_sentence)
    print(word_free)
    print(punc_free)
    print(alpha_free)
    print(stem_free)
    

    return normalized



### removing nouns from text

In [174]:
clean("""From: Ajay Kumar Gupta [mailto:Ajay.Gupta@mbl.in] 
Sent: Monday, November 19, 2018 11:55 AM
To: Reservations <Reservations@tajhotels.com>
Subject: Re: Taj Bengal Kolkata Payment link for reservation No 75693SB014044
 
Dear Sirs,
 
Please raise your final invoice on :
 
Macawber Beekay Pvt Ltd 
C/O 4th Floor KB-25,Salt Lake City 
Kolkata 700098
West Bengal
19AAECB8838L2Z0
 
Thanking you,
 
Regards,
 
Ajay Kumar Gupta""")

19, 2018 11:55 link for reservation 75693SB014044 raise your final invoice on : 4th 700098 19AAECB8838L2Z0 you,
19, 2018 11:55 link reservation 75693sb014044 raise final invoice : 4th 700098 19aaecb8838l2z0 you,
19 2018 1155 link reservation 75693sb014044 raise final invoice  4th 700098 19aaecb8838l2z0 you
link reservation raise final invoice you
link reserv rais final invoic you


'link reservation raise final invoice you'

In [80]:
remove_nouns(data['content'][0])

'Dear As per our conversation a while ago, please book and confirm 02 Executive room at from 24/26 2018 for two nights.'

In [81]:
textFeatures[0]

'dear rohit per convers ago plea book confirm execut room taj gateway gang decemb two night'

In [141]:
## text transform 
textFeatures = data['content'].copy()
textFeatures = textFeatures.apply(clean)
vectorizer =  TfidfVectorizer(token_pattern=r"(?u)\b\w+\b", stop_words=None,  analyzer='word', norm=None)#TfidfVectorizer("english") 
features = vectorizer.fit_transform(textFeatures)

features_train, features_test, labels_train, labels_test = train_test_split(features, data['class_to_exec'], test_size=0.2, random_state=111)

In [135]:
import gensim

dictionary = gensim.corpora.Dictionary(trn_set)
dictionary.filter_extremes(no_below=20, no_above=0.1)

print(dictionary)

c:\users\lokesh\appdata\local\programs\python\python36\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


TypeError: doc2bow expects an array of unicode tokens on input, not a single string

In [134]:
trn_set = pd.DataFrame(' '.join(textFeatures.as_matrix()).split())[0].value_counts().keys()

c:\users\lokesh\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


In [155]:
vectorizer.use_idf

True

In [149]:
help(vectorizer)

Help on TfidfVectorizer in module sklearn.feature_extraction.text object:

class TfidfVectorizer(CountVectorizer)
 |  Convert a collection of raw documents to a matrix of TF-IDF features.
 |  
 |  Equivalent to CountVectorizer followed by TfidfTransformer.
 |  
 |  Read more in the :ref:`User Guide <text_feature_extraction>`.
 |  
 |  Parameters
 |  ----------
 |  input : string {'filename', 'file', 'content'}
 |      If 'filename', the sequence passed as an argument to fit is
 |      expected to be a list of filenames that need reading to fetch
 |      the raw content to analyze.
 |  
 |      If 'file', the sequence items must have a 'read' method (file-like
 |      object) that is called to fetch the bytes in memory.
 |  
 |      Otherwise the input is expected to be the sequence strings or
 |      bytes items are expected to be analyzed directly.
 |  
 |  encoding : string, 'utf-8' by default.
 |      If bytes or files are given to analyze, this encoding is used to
 |      decode.
 

In [110]:
from sklearn.naive_bayes import MultinomialNB

mnb = MultinomialNB(alpha=0.2)
mnb.fit(features_train, labels_train)
prediction = mnb.predict(features_test)
accuracy_score(labels_test,prediction)

0.8

In [179]:
## grid serach CV for tuning hyperparameters
import sklearn.model_selection as modsel
param_grid_ = {'C': [1e-5, 1e-3, 1e-1, 1e0, 1e1, 1e2]}

tfidf_search = modsel.GridSearchCV(LogisticRegression(), cv=5, param_grid=param_grid_)
tfidf_search.fit(features_train, labels_train)

c:\users\lokesh\appdata\local\programs\python\python36\lib\site-packages\sklearn\model_selection\_split.py:626: Warning: The least populated class in y has only 2 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)
c:\users\lokesh\appdata\local\programs\python\python36\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\lokesh\appdata\local\programs\python\python36\lib\site-packages\sklearn\linear_model\logistic.py:459: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
c:\users\lokesh\appdata\local\programs\python\python36\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from 

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'C': [1e-05, 0.001, 0.1, 1.0, 10.0, 100.0]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [176]:
pre = tfidf_search.predict(features_test)
accuracy_score(labels_test,pre)

0.8

In [111]:
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC

svc = SVC(kernel='sigmoid', gamma=1.0)
svc.fit(features_train, labels_train)
prediction = svc.predict(features_test)
accuracy_score(labels_test,prediction)

0.6

In [112]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
lr.fit(features_train, labels_train)
prediction = lr.predict(features_test)
accuracy_score(labels_test,prediction)

c:\users\lokesh\appdata\local\programs\python\python36\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\lokesh\appdata\local\programs\python\python36\lib\site-packages\sklearn\linear_model\logistic.py:459: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


0.6

In [90]:
help(vectorizer)

Help on TfidfVectorizer in module sklearn.feature_extraction.text object:

class TfidfVectorizer(CountVectorizer)
 |  Convert a collection of raw documents to a matrix of TF-IDF features.
 |  
 |  Equivalent to CountVectorizer followed by TfidfTransformer.
 |  
 |  Read more in the :ref:`User Guide <text_feature_extraction>`.
 |  
 |  Parameters
 |  ----------
 |  input : string {'filename', 'file', 'content'}
 |      If 'filename', the sequence passed as an argument to fit is
 |      expected to be a list of filenames that need reading to fetch
 |      the raw content to analyze.
 |  
 |      If 'file', the sequence items must have a 'read' method (file-like
 |      object) that is called to fetch the bytes in memory.
 |  
 |      Otherwise the input is expected to be the sequence strings or
 |      bytes items are expected to be analyzed directly.
 |  
 |  encoding : string, 'utf-8' by default.
 |      If bytes or files are given to analyze, this encoding is used to
 |      decode.
 

In [93]:
features.toarray().shape

(24, 183)

In [161]:
1e1*0.1

1.0